In [1]:
%run stdPackages.ipynb
from lpEnergyModels.mBasic import * # Load everything from the mBasic module

*Note: "`self`" is a placeholder that refers to the class MBasic or a specific instance of the class.*

# How to do a simple extension

The notebook shows how to do a simple extension on an existing model class. Specifically, we explain the extension `MBasicEmCap(MBasic)` that adds an emissions cap to a simple power system model. This is a simple class extension in the sense that the existing model structure from `MBasic` is unchanged: The variables and variational constraints that we added to `MBasic` does not change in the current extension, we only add new components on top of it.

## Model

The `MBasic` model solves for vectors of electricity generation ($E_i$ = `generation`) and demand levels ($D_{ic}$ = `demand`) by solving:
$$\begin{align}
    &\max\mbox{ }\sum_{ic}u_{ic} D_{ic} - \sum_{i} c_i E_i \tag{1A} \\
    &\text{s.t. }\sum_{ic} D_{ic} = \sum_i E_i \tag{1B} \\
    & E_i\in[0,q_i], \quad \text{and} \quad D_{ic} \in[0, L_{ic}], \tag{1C}
\end{align}$$
and where the marginal costs of generation, $E_i$, are computed from:
$$\begin{align}
    c_i = c_i^{oth} + \sum_f \mu_{f,i} (p_f + \sum_{em} \phi_{f,em}\cdot\tau_{em}) \tag{2}, 
\end{align}$$
and where $i$ is the set of generators, $f$ the set of fuel types, $em$ the set of emission types, $ic$ the set of consumers, $p_f$ is the price of fuels, $\phi_{f,em}$ the emission intensity, $c_i^{oth}$ captures "other" variable and operating costs than fuel and emisisons related costs, $\mu_{f,i}$ indicates fuel intensities for generator $i$, $\tau_{em}$ a tax on emissions, $q_i$ the installed generating capacity, $u_{ic}$ the marginal willingness to pay for consumers, $L_{ic}$ the total demand if the price does not exceed their marginal willingness to pay.

The `MBasicEmcap` model adds a variational constraint, namely:

$$\begin{align}
    \sum_{i} E_i \left(\sum_f\mu_{f,i} \phi_{f,em}\right) \leq \overline{M}_{em}, \tag{3}
\end{align}$$
where $\overline{M}_{em}$ is a cap on emissions and $\left(\sum_f\mu_{f,i} \phi_{f,em}\right)$ measures the emission intensity of a generator, i.e. the ton of emissions that comes with 1 GJ of output generated. 

As always, we refer to the "augmented form" of the model as the one that stacks constraints and variables to the format:
$$\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{4A}\\ 
    &A_{ub}\times x \leq b_{ub} \tag{4B}\\ 
    &A_{eq}\times x  = b_{eq} \tag{4C}\\ 
    &l\leq x\leq u, \tag{4D}
\end{align}$$
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

## Class extension 

This simple class extension can be added through adding two small methods. Here is a copy of the entire class:

We add a new variational constraint by first declaring it in the `self.sys.ub` dictionary:
```python 
class MBasicEmCap(MBasic):
	# Add additional constraint:
	def initArgs_ub(self):
		""" self.sys.ub dictionary"""
		self.sys.ub.update({'emCap': self.db('idxEm')})

	def initArgsUb_emCap(self):
		self.sys.lazyA('emCap2Gen', series = plantEmissionIntensity(self.db('uFuel'),self.db('uEm')),  v = 'generation', constr = 'emCap',attr='ub')
		self.sys.lp['b_ub'][('emCap','emCap')] = self.db('emCap')
```

*Note that because the `MBasic` model did not have any inequality constraints, we add the new methods here without calling `super()`.*

Note that due to the structure of our model classes, the `self.compile` method automatically executes these methods at the right time - we do not have to add them anywhere else:  `self.initArgs_ub` is always called and `self.initArgsUb_emCap` is called *because* we have specified that the model now contains a variational constraint named `emCap`.

The following lines of code loads data and builds the model and its components:

In [2]:
testData = os.path.join(_d['data'],'EX_MBasic.xlsx')
data = pyDbs.ExcelSymbolLoader(testData)() # read data in a dict 
m = MBasicEmCap() # initialize model
[m.db.__setitem__(k, v) for k,v in data.items() if k!='__meta__']; # add data to model database
m.compile(); # build LP structure

The two methods specify how to add the constraint to the underlying `self.sys`. 

The first method `self.initArgs_ub` adds that we now have a new variational constraint named `emCap` and that this is defined on the emission index `idxEm`:

In [3]:
m.sys.ub # look at ub constraints and their relevant domains

{'emCap': Index(['CO2'], dtype='object', name='idxEm')}

The second method specifies parameters for "augmented linear programming form". The first line of code specifies that the variable `generation` enters the constraint `emCap` proportional to the relevant generators' emission intensity. The emission intensity is computed calling the local method `plantEmissionIntensity(uFuel, uEm)` that computes this sum $\left(\sum_f\mu_{f,i} \phi_{f,em}\right)$. 

In [4]:
plantEmissionIntensity(m.db('uFuel'), m.db('uEm'))

idxGen   idxEm
biomass  CO2      0.0000
coal     CO2      0.2632
natgas   CO2      0.1288
nuclear  CO2      0.0030
solar    CO2      0.0000
wind     CO2      0.0000
dtype: float64

We add this to the model using the `self.sys.lazyA` method (see documentation for `LPSys` [here](https://github.com/ChampionApe/symMaps/blob/main/docs_LPSys.ipynb)) which automatically checks a couple of things when adding it to the matrix `A_ub` (augmented form). We can check that the argument has a valid form by calling:

In [7]:
m.sys.validateA('emCap2Gen','ub')

This looks for the element `emCap2Gen` (that we just added) in the `self.sys.lp['A_ub']` dictionary that contains all coefficients that goes into the `A_ub` matrix. If the component does not have the right shape or is inconsistent with the domains we have declared under `self.sys.ub`, this raises an error and points to the problem. As it does not raise an error, everything is okay here.

Note that we can run a similar check on *all* structures that we have added so far by calling (same idea, it raises an error if there is an issue):

In [8]:
m.sys.validateAll()

The final line adds the part that goes to the constant `b_ub`: This is simply the emissions cap `emCap` that we can find in the data. We can validate that this structure is consistent as well:

In [12]:
m.sys.validateV('emCap', 'b_ub')